In [1]:
import time
import datetime
import pandas as pd
import random
import json
from urllib.request import urlopen

In [6]:
url = "http://www.shfe.com.cn/data/dailydata/kx/pm20180416.dat"
page = json.load(urlopen(url))

In [4]:
now_date = datetime.datetime(2018,4,16)
split_date = now_date.strftime('%Y-%m-%d').split('-')

In [7]:
page

{'o_code': 0,
 'o_cursor': [{'CJ1': 422189,
   'CJ1_CHG': -4840,
   'CJ2': 413442,
   'CJ2_CHG': -1758,
   'CJ3': 415299,
   'CJ3_CHG': -2079,
   'INSTRUMENTID': 'cuall                         ',
   'PARTICIPANTABBR1': '期货公司  ',
   'PARTICIPANTABBR2': '期货公司  ',
   'PARTICIPANTABBR3': '期货公司  ',
   'PARTICIPANTID1': '期货公司  ',
   'PARTICIPANTID2': '期货公司  ',
   'PARTICIPANTID3': '期货公司  ',
   'PRODUCTNAME': '铜                  ',
   'PRODUCTSORTNO': 10,
   'RANK': -1},
  {'CJ1': 941,
   'CJ1_CHG': -1082,
   'CJ2': 5662,
   'CJ2_CHG': -15,
   'CJ3': 3805,
   'CJ3_CHG': 306,
   'INSTRUMENTID': 'cuall                         ',
   'PARTICIPANTABBR1': '非期货公司',
   'PARTICIPANTABBR2': '非期货公司',
   'PARTICIPANTABBR3': '非期货公司',
   'PARTICIPANTID1': '非期货公司',
   'PARTICIPANTID2': '非期货公司',
   'PARTICIPANTID3': '非期货公司',
   'PRODUCTNAME': '铜                  ',
   'PRODUCTSORTNO': 10,
   'RANK': 0},
  {'CJ1': 1245,
   'CJ1_CHG': -220,
   'CJ2': 6030,
   'CJ2_CHG': 250,
   'CJ3': 2530,
   'CJ3_CHG': -75,


In [4]:
data = pd.DataFrame(columns=(0,1,2,3,4,5,6,7,8,9,10,11))
all_col_insert = True
other_col_insert = True

last_type = 'null'

test_count = 0
length = 0

now_date = datetime.datetime(2018,4,16)

while now_date < datetime.datetime.now():
    
    split_date = now_date.strftime('%Y-%m-%d').split('-')
    
    try:
        url = "http://www.shfe.com.cn/data/dailydata/kx/pm" + split_date[0]+split_date[1]+split_date[2] + ".dat"
        page = json.load(urlopen(url))
        time.sleep(random.randrange(1,3))
        
        for i in page['o_cursor']:

            length = len(data)

            this_type = i['INSTRUMENTID'].strip()

            if last_type != this_type and last_type!='null':

                data.to_csv('../各交易所期貨持倉量/上期所/更新/'+last_type + "_" + split_date[0]+split_date[1]+split_date[2]+".csv",encoding='utf-8-sig')

                data = pd.DataFrame(columns=(0,1,2,3,4,5,6,7,8,9,10,11))

                last_type = this_type

                all_col_insert=True
                other_col_insert=True

                if "all" in this_type:

                    insert_data = insert_all(i, other_col_insert)            
                    length = len(data)
                    data.loc[length] = insert_data

                    other_col_insert = False
                    continue

                else: 
                    insert_data = contract(i, all_col_insert, length)

                    length = len(data)
                    data.loc[length] = insert_data

                    all_col_insert=False     
                    continue

            if "all" in this_type:
                insert_data = insert_all(i, other_col_insert)
                length = len(data)
                data.loc[length] = insert_data

                last_type = this_type
                continue            

            insert_data = contract(i, all_col_insert, length)  
            length = len(data)
            data.loc[length] = insert_data

            last_type = this_type

        now_date = now_date + datetime.timedelta(days = 1)
    
    except:
        now_date = now_date + datetime.timedelta(days = 1)

In [2]:
def insert_all(page_data,new_col_insert):
    if new_col_insert:
            col = ['会员类别','总成交量','比上交易日增减','会员类别','总持买单量','比上交易日增减',\
                   '会员类别','总持卖单量','比上交易日增减',None,None,None]
            data.loc[0] = col
              
    col_data = [page_data['PARTICIPANTABBR1'].strip(),page_data['CJ1'],page_data['CJ1_CHG'],\
                page_data['PARTICIPANTABBR2'].strip(),page_data['CJ2'],page_data['CJ2_CHG'],\
                page_data['PARTICIPANTABBR3'].strip(),page_data['CJ3'],page_data['CJ3_CHG']]
    col_data.extend((None,None,None))
    return col_data

In [3]:
def contract(page_data,new_col_insert,length):
        
    if page_data['RANK'] == 999 :
        col_data = ['合计',page_data['CJ1'],page_data['CJ1_CHG'],\
                    '合计',page_data['CJ2'],page_data['CJ2_CHG'],\
                    '合计',page_data['CJ3'],page_data['CJ3_CHG']]
        col_data.extend((None,None,None))
        data.loc[length] = col_data
        
    else:
        if new_col_insert:
            col = ['名次','期货公司会员简称','成交量','比上交易日增减','名次','期货公司会员简称','持买单量','比上交易日增减','名次','期货公司会员简称','持卖单量','比上交易日增减/变化']
            data.loc[0] = col
        
        col_data = [page_data['RANK'],page_data['PARTICIPANTABBR1'].strip(),page_data['CJ1'],page_data['CJ1_CHG'],\
                    page_data['RANK'],page_data['PARTICIPANTABBR2'].strip(),page_data['CJ2'],page_data['CJ2_CHG'],\
                    page_data['RANK'],page_data['PARTICIPANTABBR3'].strip(),page_data['CJ3'],page_data['CJ3_CHG']]
        return col_data